In [1]:
#Conversting JSONL to CSV
import json
import csv


data = []
with open('all_messages.jsonl') as f:
    for line in f:
        data.append(json.loads(line))


with open('all_messages.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(data[0].keys())
    
    for d in data:
        writer.writerow(d.values())

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('all_messages.csv')
df2 = df.copy

In [12]:
df['user'] = df['user'].str.replace('[\[\]]', '', regex=True)
df['customer'] = df['customer'].str.replace('[\[\]]', '', regex=True)


In [7]:
df.shape

(14896, 2)

In [13]:
import numpy as np

# Replace empty values with NaN
df['user'] = df['user'].replace(r'^\s*$', np.nan, regex=True)



In [14]:
df = df.dropna(subset=['user'])

In [16]:
#Removing questions generated by bots

df = df[~df['user'].str.contains("bot example")]
df = df[~df['user'].str.contains("bot example2")]


In [ ]:
df['user'].value_counts()

In [10]:
#Removing questions generated by bots
mask = ~df['user'].str.contains('bot example3')
df = df[mask]

In [22]:
import nltk

In [ ]:
from nltk import FreqDist
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

conversations = df['customer'].tolist()

# This ensures that '.' only acts as a sentence delimiter if it is followed by a whitespace, effectively ensuring abbreviations are not split across different sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Split into sentences
sentence_list = []
for conv in conversations:
    sentence_list.extend(tokenizer.tokenize(conv))

# Filter sentences to select only those that end with a '?'
question_list = [sentence for sentence in sentence_list if sentence.endswith('?')]

# Count the frequency of each question
count = Counter(question_list)

# Find the most common question
most_common_question = count.most_common(10)

print(most_common_question)

In [128]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
import re

for question, freq in most_common_question:
    escaped_question = re.escape(question)
    matched_rows = df[df['customer'].str.contains(escaped_question)]
    print('Question: ', question)
    print('Frequency: ', freq)
    print('Matched rows:')
    print(matched_rows)
    matched_rows.to_csv('matched_rows.csv', mode='a', header=False)

In [138]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter


# You might need to download the set of stop words the first time
nltk.download('punkt')
nltk.download('stopwords')

# Create a set of English stop words
stop_words = set(stopwords.words('english'))

# Tokenize the words in your 'customer' column and remove stop words
df['customer_clean'] = df['customer'].apply(lambda x: ' '.join([word for word in word_tokenize(x.lower()) if word.isalpha() and word not in stop_words]))

[nltk_data] Downloading package punkt to /Users/boyoon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boyoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Running N grams to extract the most common questions

import nltk
from collections import Counter

conversations = df['customer_clean'].tolist()
words = nltk.word_tokenize(' '.join(conversations))
fourgrams = nltk.ngrams(words, 10) #We are using 4 here since you asked for 4-grams

fourgram_freq = Counter(fourgrams)

most_common_fourgrams = fourgram_freq.most_common(10)

for grams in most_common_fourgrams: 
    print(grams)

In [ ]:
questions_df = pd.DataFrame(df['customer'].value_counts()).reset_index()
questions_df.columns = ['Question', 'Frequency']

# Adding corresponding answers to the questions in the new dataframe
questions_df['Answer'] = questions_df['Question'].map(df.set_index('customer')['user'].to_dict())

# Printing the rows
for index, row in questions_df.head(10).iterrows():
    print(f"Question: {row['Question']} \nAnswer: {row['Answer']} \nFrequency: {row['Frequency']}\n")

In [28]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Initialize the TfidfVectorizer
# vectorizer = TfidfVectorizer()  

# # Fit and transform your questions 
# tfidf_matrix = vectorizer.fit_transform(df['customer'])  

# # Compute the cosine similarity matrix 
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)  

# # Convert the cosine similarity matrix to a DataFrame 
# sim_df = pd.DataFrame(cosine_sim, index=df['customer'], columns=df['customer'])  

# # For each question, get the top 5 most similar questions 
# top_similar_questions = sim_df.apply(lambda row: row.nlargest(6).index.tolist()[1:], axis=1)  

# # Add the similar questions to your DataFrame 
# df['Similar_Questions'] = df['customer'].map(top_similar_questions.to_dict())

In [50]:
for question, freq in most_common_questions:
    # get method will return None if there is no such key
    answer = qa_pairs.get(question)  

    if answer is not None:
        print(f"Question: {question} \nAnswer: {answer} \nFrequency: {freq}")

In [30]:
import pandas as pd
from transformers import pipeline


/opt/homebrew/anaconda3/envs/OptAnalytics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
import replicate

In [32]:
import os
os.environ['REPLICATE_API_TOKEN'] = ''


In [55]:
convo10 = df_reset['user'].tolist()

In [ ]:
df_reset['user'].value_counts()

In [56]:
total_words = 0

# Iterate through each sentence in the convo10 list and count the words
for sentence in convo10:
    words = sentence.split()  # Split the sentence into words based on spaces
    total_words += len(words)  # Count the number of words in the current sentence

print(f'The total number of words in the convo10 list is: {total_words}')


The total number of words in the convo10 list is: 5005


In [ ]:
#Trying out the llama2 chat model
output= replicate.stream(         
        "meta/llama-2-70b-chat:",         
        input={             
            "prompt": "From the list of answers, show me how they were answered overall and organize them in lists" + ' '.join(second_half),
            "temperature": 0.5,
            "system_prompt": "You are a helpful assistant.",           
            "max_new_tokens": 500,         
        }     
    )  
for item in output:     
        print(item, end="")

In [ ]:
conversations[1]

In [69]:
# Import OpenAI library for GPT-3
import openai

# Initialize OpenAI API with your API key
openai.api_key = ''


In [105]:
len(conversations)

4558

In [75]:
df_w2 = pd.read_csv("w2_rows.csv")

convo_w2 = df_w2['user'].tolist()

In [77]:
len(convo_w2)

41

In [79]:
df_expedite = pd.read_csv("expedite_rows.csv")

convo_expedite = df_expedite['user'].tolist()

len(convo_expedite)

29

In [ ]:


# Generate an answer using OpenAI GPT-4
response = openai.ChatCompletion.create(
    model="gpt-4-turbo-preview", 
    messages=[{"role": "system", "content": "from the list of answers, show me how it was generally answered in a list: " + ' '.join(convo_empl)}],
    max_tokens=500
)

# Extract the summary from the response
answer = response.choices[0]['message']['content']

# Print the summary
print(answer)